In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot
#import plotly.graph_objs as go
import os
import warnings

# 파일 불러와서 열이름 바꾸기

In [2]:
##### 파일 불러오기 #####
df = pd.read_csv("C:/Users/user/Desktop/2021 빅콘테스트_데이터분석분야_퓨처스리그_ECO제주\data_processing/2_3_short_term_frgn_3_resident_pop_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'visit_pop_cnt', 'total_pop']  ##### 각자 데이터에 맞게 열이름 변경하기 #####
df

,base_date,emd_cd,visit_pop_cnt,total_pop
base_date,,,,
2018-01-31,2018-01-31,50110250,58090.5960,24419
2018-02-28,2018-02-28,50110250,69710.8394,24427
2018-03-31,2018-03-31,50110250,98026.6381,24532
2018-04-30,2018-04-30,50110250,103289.7891,24606
2018-05-31,2018-05-31,50110250,115255.9750,24715
...,...,...,...,...
2021-02-28,2021-02-28,50130620,280.6159,4173
2021-03-31,2021-03-31,50130620,1475.8404,4178
2021-04-30,2021-04-30,50130620,1283.0997,4161


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date', 'visit_pop_cnt'], 1)     ##### 각자 필요없는 열 제거 #####
data

,emd_cd,total_pop
base_date,,
2018-01-31,50110250,24419
2018-02-28,50110250,24427
2018-03-31,50110250,24532
2018-04-30,50110250,24606
2018-05-31,50110250,24715
...,...,...
2021-02-28,50130620,4173
2021-03-31,50130620,4178
2021-04-30,50130620,4161


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130570  ##### 각 행정동코드 입력 #####

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,total_pop
base_date,
2018-01-31,23959
2018-02-28,23982
2018-03-31,23963
2018-04-30,23960
2018-05-31,23948
2018-06-30,23994
2018-07-31,24033
2018-08-31,24053
2018-09-30,24110


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'total_pop': base_date
 2018-01-31    23959
 2018-02-28    23982
 2018-03-31    23963
 2018-04-30    23960
 2018-05-31    23948
 2018-06-30    23994
 2018-07-31    24033
 2018-08-31    24053
 2018-09-30    24110
 2018-10-31    24124
 2018-11-30    24164
 2018-12-31    24163
 2019-01-31    24170
 2019-02-28    24165
 2019-03-31    24138
 2019-04-30    24121
 2019-05-31    24151
 2019-06-30    24185
 2019-07-31    24159
 2019-08-31    24227
 2019-09-30    24301
 2019-10-31    24334
 2019-11-30    24380
 2019-12-31    24389
 2020-01-31    24408
 2020-02-29    24406
 2020-03-31    24432
 2020-04-30    24418
 2020-05-31    24400
 2020-06-30    24368
 2020-07-31    24389
 2020-08-31    24352
 2020-09-30    24365
 2020-10-31    24280
 2020-11-30    24237
 2020-12-31    24222
 2021-01-31    24166
 2021-02-28    24170
 2021-03-31    24145
 2021-04-30    24104
 2021-05-31    24111
 2021-06-30    24097
 Name: total_pop, dtype: int64}

# ARIMA

In [6]:
# ARIMA에 적합한 log로 바꾸는 과정
sub_df_log = np.log1p(sub_df[:])
#sub_df_float.tail()

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df_log)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 p-value 확인 -> 0.05보다 크면 아래로 내려가서 차분하기 / 작으면 차분 생략(아래 셀 생략) #####

ADF Statistic : -2.388704
p-value : 0.144950
Critical Values :
	1%: -3.615509
	5%: -2.941262
	10%: -2.609200


In [8]:
# 차분
diff_1 = sub_df_log.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))
    
##### 여기서 또 p-value 확인 -> 0.05보다 크면 얼른 단톡에 보고하기 / 작으면 아래로 쭉 진행 #####

ADF Statistic : -2.401952
p-value : 0.141186
Critical Values :
	1%: -3.610400
	5%: -2.939109
	10%: -2.608063


In [9]:
#!pip install pmdarima

In [10]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df_log,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df_log)

##### 여기서 아래쪽에 'Best model:  ARIMA(1,1,1)(0,0,0)[1] intercept' 이렇게 나오면 (1,1,1) 가져다 쓸거임 #####
##### ARIMA(1,1,1) 여기 숫자 세 개만 잘 보면 됨 !!!! #####

 ARIMA(0,2,0)(0,0,0)[1]             : AIC=-397.457, Time=0.03 sec
 ARIMA(0,2,1)(0,0,0)[1]             : AIC=-413.247, Time=0.07 sec
 ARIMA(0,2,2)(0,0,0)[1]             : AIC=-412.163, Time=0.10 sec
 ARIMA(0,2,3)(0,0,0)[1]             : AIC=-410.155, Time=0.17 sec
 ARIMA(0,2,4)(0,0,0)[1]             : AIC=-407.887, Time=0.14 sec
 ARIMA(0,2,5)(0,0,0)[1]             : AIC=-404.040, Time=0.40 sec
 ARIMA(1,2,0)(0,0,0)[1]             : AIC=-410.987, Time=0.10 sec
 ARIMA(1,2,1)(0,0,0)[1]             : AIC=-412.349, Time=0.10 sec
 ARIMA(1,2,2)(0,0,0)[1]             : AIC=-410.317, Time=0.11 sec
 ARIMA(1,2,3)(0,0,0)[1]             : AIC=-408.234, Time=0.33 sec
 ARIMA(1,2,4)(0,0,0)[1]             : AIC=-405.172, Time=0.08 sec
 ARIMA(2,2,0)(0,0,0)[1]             : AIC=-412.468, Time=0.07 sec
 ARIMA(2,2,1)(0,0,0)[1]             : AIC=-410.506, Time=0.11 sec
 ARIMA(2,2,2)(0,0,0)[1]             : AIC=-408.530, Time=0.19 sec
 ARIMA(2,2,3)(0,0,0)[1]             : AIC=-406.570, Time=0.31 sec
 ARIMA(3,2

ARIMA(order=(0, 2, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True, with_intercept=False)

In [11]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df_log.total_pop.values, order=(0,2,1))     ##### 여기 order=(1,1,1)에 위에서 나온 숫자 넣는거 #####
                                   ##### 그리고 sub_df_log.waste_cnt.values에서 가운데 waste_cnt 각자걸로 바꾸기 #####

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                   40
Model:                 ARIMA(0, 2, 1)   Log Likelihood                 208.210
Method:                       css-mle   S.D. of innovations              0.001
Date:                Sat, 11 Sep 2021   AIC                           -410.420
Time:                        01:24:23   BIC                           -405.353
Sample:                             2   HQIC                          -408.588
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -5.096e-05   1.67e-05     -3.046      0.002   -8.38e-05   -1.82e-05
ma.L1.D2.y    -1.0000        nan        nan        nan         nan         nan
                                    Roots           

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warn

# 예측하기

In [12]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

##### 맨 앞에 나온 'array([10681271.81014634, 10741037.02029268])' 이게 차례대로 7, 8월 예측값에 로그씌운거임 #####
##### 7월 : 10.05681231, 8월 : 10.28080669 #####
##### 이거 지수함수에 넣어서 원래값으로 돌려야 함 -> 각각 가져다 밑에 넣기 #####

(array([10.08894654, 10.08795799]), array([0.00126763, 0.0017927 ]), array([[10.08646204, 10.09143104],
       [10.08444437, 10.09147161]]))


In [13]:
# 이거 위에서 로그씌울 때 np.log1p 이거 써서 지수함수에 넣은 다음에 1 뺀거임
# https://rfriend.tistory.com/295
# 궁금하면 이 링크 들어가서 읽어보세요 질문받기 싫어서 미리 써놓음
# 밑에 어떻게 되는지 예제코드도 넣었으니까 제발 질문 안해주길 ㅎ

print( np.exp(10.08894654) - 1 )   ##### 괄호 안에 7월 숫자 넣기 #####
print( np.exp(10.08795799) - 1 )   ##### 괄호 안에 8월 숫자 넣기 #####

##### 이 값 가져다 엑셀파일에 입력하기 #####

24074.41658136231
24050.628588048385
